In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import ImageFolder
from torchvision.models import resnet50, ResNet50_Weights
from sklearn.metrics import f1_score
import time
import numpy as np
from sklearn.metrics import f1_score
import time
import matplotlib.pyplot as plt


# Set device
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(DEVICE)

cuda


In [7]:
##########################
### SETTINGS
##########################
# Hyperparameters
RANDOM_SEED = 1
LEARNING_RATE = 0.0001
BATCH_SIZE = 64
NUM_EPOCHS = 10
GRAYSCALE = False
DATA_DIR = "./dataset"

In [8]:
# Transforms
transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Custom Dataset class
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.image_folder_dataset = ImageFolder(root=self.data_dir, transform=self.transform)

    def __len__(self):
        return len(self.image_folder_dataset)

    def __getitem__(self, idx):
        return self.image_folder_dataset[idx]

    @property
    def classes(self):
        return self.image_folder_dataset.classes

# Load dataset and split into train, validation, test
custom_dataset = CustomDataset(DATA_DIR, transform=transform)
train_size = int(0.6 * len(custom_dataset))
val_size = int(0.2 * len(custom_dataset))
test_size = len(custom_dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(custom_dataset, [train_size, val_size, test_size])

# DataLoaders
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

print( len(custom_dataset.classes))
print( len(train_dataloader))
print( len(val_dataloader))
print( len(test_dataloader))

# Model definition

12
46
16
16


In [9]:
model = resnet50(weights=ResNet50_Weights.DEFAULT)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, len(custom_dataset.classes))
model.to(DEVICE)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [10]:
# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [ ]:
def compute_metrics(model, data_loader, device):
    model.eval()
    correct_pred = 0
    total_examples = 0
    all_predictions = []
    all_targets = []
    total_loss = 0

    with torch.no_grad():
        for features, targets in data_loader:
            features = features.to(device)
            targets = targets.to(device)
            logits = model(features)
            loss = F.cross_entropy(logits, targets)
            total_loss += loss.item()

            _, predicted_labels = torch.max(logits, 1)
            correct_pred += (predicted_labels == targets).sum().item()
            total_examples += targets.size(0)

            all_predictions.extend(predicted_labels.cpu().numpy())
            all_targets.extend(targets.cpu().numpy())

    accuracy = correct_pred / total_examples * 100
    average_loss = total_loss / total_examples
    f1 = f1_score(all_targets, all_predictions, average='weighted')
    
    return accuracy, average_loss, f1

start_time = time.time()

train_losses = []
train_accuracies = []
train_f1_scores = []

valid_losses = []
valid_accuracies = []
valid_f1_scores = []

test_losses = []
test_accuracies = []
test_f1_scores = []

for epoch in range(NUM_EPOCHS):
    model.train()
    epoch_train_loss = 0.0
    
    for batch_idx, (features, targets) in enumerate(train_dataloader):
        features = features.to(DEVICE)
        targets = targets.to(DEVICE)
        optimizer.zero_grad()
        logits = model(features)
        loss = F.cross_entropy(logits, targets)
        loss.backward()
        optimizer.step()
        epoch_train_loss += loss.item()
        
        if not batch_idx % 120:
            print(f'Epoch: {epoch+1:03d}/{NUM_EPOCHS:03d} | '
                  f'Batch {batch_idx:03d}/{len(train_dataloader):03d} |' 
                  f' Cost: {loss:.4f}')
    
    train_loss = epoch_train_loss / len(train_dataloader)
    train_acc, _, train_f1 = compute_metrics(model, train_dataloader, DEVICE)
    val_acc, val_loss, val_f1 = compute_metrics(model, val_dataloader, DEVICE)
    test_acc, test_loss, test_f1 = compute_metrics(model, test_dataloader, DEVICE)

    print(f'Epoch: {epoch+1:03d}/{NUM_EPOCHS:03d} '
          f'Train Acc.: {train_acc:.2f}% | Train Loss: {train_loss:.4f} | Train F1: {train_f1:.4f} '
          f'| Validation Acc.: {val_acc:.2f}% | Validation Loss: {val_loss:.4f} | Validation F1: {val_f1:.4f}'
          f'| Test Acc.: {test_acc:.2f}% | Test Loss: {test_loss:.4f} | Test F1: {test_f1:.4f}')

    train_losses.append(train_loss)
    train_accuracies.append(train_acc)
    train_f1_scores.append(train_f1)

    valid_losses.append(val_loss)
    valid_accuracies.append(val_acc)
    valid_f1_scores.append(val_f1)

    test_losses.append(test_loss)
    test_accuracies.append(test_acc)
    test_f1_scores.append(test_f1)

    elapsed = (time.time() - start_time) / 60
    print(f'Time elapsed: {elapsed:.2f} min')

In [ ]:
# Show Summary
print("Training Summary:")
print("Train Loss:", train_losses[-1])
print("Train Accuracy:", train_accuracies[-1])
print("Train F1 Score:", train_f1_scores[-1])

print("\nValidation Summary:")
print("Validation Loss:", valid_losses[-1])
print("Validation Accuracy:", valid_accuracies[-1])
print("Validation F1 Score:", valid_f1_scores[-1])

print("\nTest Summary:")
print("Test Loss:", test_losses[-1])
print("Test Accuracy:", test_accuracies[-1])
print("Test F1 Score:", test_f1_scores[-1])

# Plotting History
plt.figure(figsize=(12, 4))

plt.subplot(1, 3, 1)
plt.plot(train_losses, label='Train Loss')
plt.plot(valid_losses, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss History')
plt.legend()

plt.subplot(1, 3, 2)
plt.plot(train_accuracies, label='Train Accuracy')
plt.plot(valid_accuracies, label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Accuracy History')
plt.legend

# Saving History Summary
history_summary = {
    "train_losses": train_losses,
    "valid_losses": valid_losses,
    "test_losses": test_losses,
    "train_accuracies": train_accuracies,
    "valid_accuracies": valid_accuracies,
    "test_accuracies": test_accuracies,
    "train_f1_scores": train_f1_scores,
    "valid_f1_scores": valid_f1_scores,
    "test_f1_scores": test_f1_scores
}

# Save to a file
import json

with open("history_summary.json", "w") as outfile:
    json.dump(history_summary, outfile)


In [ ]:
# Training loop
# def compute_metrics(model, data_loader, device):
#     model.eval()
#     correct_pred = 0
#     total_examples = 0
#     all_predictions = []
#     all_targets = []
#     total_loss = 0

#     with torch.no_grad():
#         for features, targets in data_loader:
#             features = features.to(device)
#             targets = targets.to(device)
#             logits = model(features)
#             loss = F.cross_entropy(logits, targets)
#             total_loss += loss.item()

#             _, predicted_labels = torch.max(logits, 1)
#             correct_pred += (predicted_labels == targets).sum().item()
#             total_examples += targets.size(0)

#             all_predictions.extend(predicted_labels.cpu().numpy())
#             all_targets.extend(targets.cpu().numpy())

#     accuracy = correct_pred / total_examples * 100
#     average_loss = total_loss / total_examples
#     f1 = f1_score(all_targets, all_predictions, average='weighted')
    
#     return accuracy, average_loss, f1

# start_time = time.time()

# for epoch in range(NUM_EPOCHS):
#     model.train()
#     epoch_train_loss = 0.0
    
#     for batch_idx, (features, targets) in enumerate(train_dataloader):
#         features = features.to(DEVICE)
#         targets = targets.to(DEVICE)
#         optimizer.zero_grad()
#         logits = model(features)
#         loss = F.cross_entropy(logits, targets)
#         loss.backward()
#         optimizer.step()
#         epoch_train_loss += loss.item()
        
#         if not batch_idx % 120:
#             print(f'Epoch: {epoch+1:03d}/{NUM_EPOCHS:03d} | '
#                   f'Batch {batch_idx:03d}/{len(train_dataloader):03d} |' 
#                   f' Cost: {loss:.4f}')
    
#     train_loss = epoch_train_loss / len(train_dataloader)
#     train_acc, _, train_f1 = compute_metrics(model, train_dataloader, DEVICE)
#     val_acc, val_loss, val_f1 = compute_metrics(model, val_dataloader, DEVICE)
#     test_acc, test_loss, test_f1 = compute_metrics(model, test_dataloader, DEVICE)

#     print(f'Epoch: {epoch+1:03d}/{NUM_EPOCHS:03d} '
#           f'Train Acc.: {train_acc:.2f}% | Train Loss: {train_loss:.4f} | Train F1: {train_f1:.4f} '
#           f'| Validation Acc.: {val_acc:.2f}% | Validation Loss: {val_loss:.4f} | Validation F1: {val_f1:.4f}'
#           f'| Test Acc.: {test_acc:.2f}% | Test Loss: {test_loss:.4f} | Test F1: {test_f1:.4f}')

#     elapsed = (time.time() - start_time) / 60
#     print(f'Time elapsed: {elapsed:.2f} min')

# elapsed = (time.time() - start_time) / 60

In [ ]:
# def conv3x3(in_planes, out_planes, stride=1):
#     """3x3 convolution with padding"""
#     return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
#                      padding=1, bias=False)


# class Bottleneck(nn.Module):
#     expansion = 4

#     def __init__(self, inplanes, planes, stride=1, downsample=None):
#         super(Bottleneck, self).__init__()
#         self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, bias=False)
#         self.bn1 = nn.BatchNorm2d(planes)
#         self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride,
#                                padding=1, bias=False)
#         self.bn2 = nn.BatchNorm2d(planes)
#         self.conv3 = nn.Conv2d(planes, planes * 4, kernel_size=1, bias=False)
#         self.bn3 = nn.BatchNorm2d(planes * 4)
#         self.relu = nn.ReLU(inplace=True)
#         self.downsample = downsample
#         self.stride = stride

#     def forward(self, x):
#         residual = x

#         out = self.conv1(x)
#         out = self.bn1(out)
#         out = self.relu(out)

#         out = self.conv2(out)
#         out = self.bn2(out)
#         out = self.relu(out)

#         out = self.conv3(out)
#         out = self.bn3(out)

#         if self.downsample is not None:
#             residual = self.downsample(x)

#         out += residual
#         out = self.relu(out)

#         return out




# class ResNet(nn.Module):

#     def __init__(self, block, layers, num_classes, grayscale):
#         self.inplanes = 64
#         if grayscale:
#             in_dim = 1
#         else:
#             in_dim = 3
#         super(ResNet, self).__init__()
#         self.conv1 = nn.Conv2d(in_dim, 64, kernel_size=7, stride=2, padding=3,
#                                bias=False)
#         self.bn1 = nn.BatchNorm2d(64)
#         self.relu = nn.ReLU(inplace=True)
#         self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
#         self.layer1 = self._make_layer(block, 64, layers[0])
#         self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
#         self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
#         self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
#         self.avgpool = nn.AvgPool2d(7, stride=1, padding=2)
#         #self.fc = nn.Linear(2048 * block.expansion, num_classes)
#         self.fc = nn.Linear(2048, num_classes)

#         for m in self.modules():
#             if isinstance(m, nn.Conv2d):
#                 n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
#                 m.weight.data.normal_(0, (2. / n)**.5)
#             elif isinstance(m, nn.BatchNorm2d):
#                 m.weight.data.fill_(1)
#                 m.bias.data.zero_()

#     def _make_layer(self, block, planes, blocks, stride=1):
#         downsample = None
#         if stride != 1 or self.inplanes != planes * block.expansion:
#             downsample = nn.Sequential(
#                 nn.Conv2d(self.inplanes, planes * block.expansion,
#                           kernel_size=1, stride=stride, bias=False),
#                 nn.BatchNorm2d(planes * block.expansion),
#             )

#         layers = []
#         layers.append(block(self.inplanes, planes, stride, downsample))
#         self.inplanes = planes * block.expansion
#         for i in range(1, blocks):
#             layers.append(block(self.inplanes, planes))

#         return nn.Sequential(*layers)

#     def forward(self, x):
#         x = self.conv1(x)
#         x = self.bn1(x)
#         x = self.relu(x)
#         x = self.maxpool(x)

#         x = self.layer1(x)
#         x = self.layer2(x)
#         x = self.layer3(x)
#         x = self.layer4(x)

#         #x = self.avgpool(x)
#         x = x.view(x.size(0), -1)
#         logits = self.fc(x)
#         probas = F.softmax(logits, dim=1)
#         return logits, probas



# def resnet101(num_classes, grayscale):
#     """Constructs a ResNet-101 model."""
#     model = ResNet(block=Bottleneck, 
#                    layers=[3, 4, 23, 3],
#                    num_classes=NUM_CLASSES,
#                    grayscale=grayscale)
#     return model